In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
import pandas as pd
from datetime import datetime

# DoDoMoa SQL

In [2]:
db_connection_str = 'mysql+pymysql://root@localhost:3306/dash_test'
db_connection = create_engine(db_connection_str)
conn = db_connection.connect()

dodomoa_info = pd.read_csv('./data/keywordExtraction2ForSQL.csv',index_col=0)


### backend_dodomoabookinfo


In [ ]:

backend_dodomoabookinfo = dodomoa_info[['도서명','저자','출판사','ISBN','주제분류번호','등록일자','이미지주소']]
# dodomoa_info['ISBN'] = dodomoa_info['ISBN'].astype(int)
backend_dodomoabookinfo.head(1)
backend_dodomoabookinfo['출판사'] = backend_dodomoabookinfo['출판사'].fillna('-')


backend_dodomoabookinfo.to_sql(name='backend_dodomoabookinfo', con=db_connection, if_exists='append',index=False)


### backend_dodomoakeyword v1


In [ ]:

backend_dodomoakeyword = dodomoa_info.drop(columns=['도서명','저자','출판사','주제분류번호','등록일자','이미지주소','keywords'])

# Col 이름변경
k = [f'key{i}' for i in range(39)]
col = ['ISBN']
col.extend(k)
backend_dodomoakeyword.columns = col


backend_dodomoakeyword.to_sql(name='backend_dodomoakeyword', con=db_connection, if_exists='append',index=False)

### backend_dodomoakeyword v2 

In [35]:
import numpy as np
keyword = list(map(lambda x : str(x), list(range(39))))
backend_dodomoakeyword2 = dodomoa_info[keyword]

def joinListtoStr(ist:list) -> str:
    return " ".join(ist)

result = list(map(joinListtoStr,backend_dodomoakeyword2.astype(str).values.tolist()))
result = list(map(lambda x : x.replace('nan',''), result))


In [ ]:
tosql = dodomoa_info[['ISBN']]
tosql['keyword'] = result

tosql
tosql.to_sql(name='backend_dodomoakeyword2', con=db_connection, if_exists='append',index=False)

### backend_dodomoalibinfo

**도서관별 책 목록 추출**

In [ ]:
# #연습용
# filelist = os.listdir('./data/libbooks/')
# k = pd.read_csv(f'./data/libbooks/{filelist[2]}',encoding='cp949',index_col=0)
# k = k.dropna(subset='주제분류번호')
# k = k.query('등록일자 > "2018-01-01"')


In [ ]:
import os

filelist = os.listdir('./data/libbooks/')


result = []
for num, fileName in enumerate(filelist) :
    name = ['영등포','송파','강서','마포','정독','강남','고덕','동작','구로','동대문','용산','양천','도봉','남산','종로','마포평생','개포','노원','고척','남산','서대문','강동'] 
    k = pd.read_csv(f'./data/libbooks/{fileName}',encoding='cp949',index_col=0)
    k = k.dropna(subset='주제분류번호')
    k = k.query('등록일자 > "2018-01-01"')
    k005 = k[k['주제분류번호'].str.contains('005.').replace(float('nan'),False)]
    k004 = k[k['주제분류번호'].str.contains('004.').replace(float('nan'),False)]
    val = pd.concat([k004,k005])[['ISBN']]
    val['지역'] = name[num]
    result.append(val)

booklist = pd.concat(result).reset_index(drop=True)

booklist.head(3)

In [46]:
booklist.to_sql(name='backend_dodomoalibinfo', con=db_connection, if_exists='append',index=False)

25416

In [ ]:
#pymysql 연결
import pymysql
import pandas as pd
conn=pymysql.connect(host='localhost',port=int(3306),user='root',passwd='',db='dash_test')
cursor = conn.cursor(pymysql.cursors.DictCursor)



### Bookinfo 제거 및 재작성

In [ ]:
cursor.execute('''DROP TABLE backend_dodomoabookinfo''')
result = cursor.fetchall()
libinfo = pd.DataFrame(result)
libinfo

In [ ]:
cursor.execute('''CREATE TABLE backend_dodomoabookinfo (
    id INT AUTO_INCREMENT NOT NULL ,
    ISBN BIGINT NOT NULL ,
    도서명 VARChar(180),
    저자 VARChar(240),
    출판사 VARChar(50),
    주제분류번호 VARChar(20),
    등록일자 Date,
    이미지주소 VARChar(100),
    PRIMARY KEY (id, ISBN)
) ENGINE=innoDB''')
result = cursor.fetchall()
libinfo = pd.DataFrame(result)
libinfo

In [ ]:
newdf = pd.read_csv('./data/tempdata.csv',index_col=0)
newdff = newdf[['도서명','저자','출판사','ISBN','주제분류번호','등록일자','이미지주소']]

newdff['출판사'] = newdff['출판사'].fillna('-')
newdff['주제분류번호'] = '00'+ newdff['주제분류번호'].astype(str)

In [ ]:
newdff.to_sql(name='backend_dodomoabookinfo', con=db_connection, if_exists='append',index=False)